<a href="https://colab.research.google.com/github/Erike-Simon/CESAR-AED/blob/main/ProcDados_spark_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=58902ad9a4a5e696a32b2cb2197adb71893392b8f751fb63907ef3d67a4d1b0a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark import SparkConf, SparkContext

In [ ]:
# Criando um cluster local com 1 worker e a quantidade de cores igual a quantidade de cores de CPU disponíveis
# local[*]
#
# Criando um cluster local com 1 worker e a quantidade de cores igual a 8
# local[8]
#
# Criando um cluster local com 2 workers, 1 cores por worker e 3GB de RAM por worker
# local-cluster[2, 1, 3072]

# conf = SparkConf().setMaster("local[*]")
# conf = SparkConf().setMaster("local[8]")
conf = SparkConf().setMaster("local-cluster[2, 1, 3072]")
sc = SparkContext(conf=conf)
sc

<SparkContext master=local-cluster[2, 1, 3072] appName=pyspark-shell>

In [ ]:
# Comando para desativar os recursos do spark
sc.stop()

In [ ]:
!head 'drive/MyDrive/data/ml-25m/ratings.csv'

userId,movieId,rating,timestamp
1,296,5.0,1147880044
1,306,3.5,1147868817
1,307,5.0,1147868828
1,665,5.0,1147878820
1,899,3.5,1147868510
1,1088,4.0,1147868495
1,1175,3.5,1147868826
1,1217,3.5,1147878326
1,1237,5.0,1147868839


In [ ]:
ROOT_DATA_PATH = 'drive/MyDrive/data'

In [ ]:
RATINGS_CSV_PATH = f'{ROOT_DATA_PATH}/ml-25m/ratings.csv'

## Qual é o total de avaliações por valor de rating? (countByValue)

In [ ]:
%%time

file_csv = sc.textFile(RATINGS_CSV_PATH)
header = file_csv.first()
ratings = file_csv.filter(lambda row: row != header)
rating_values = ratings.map(lambda row: row.split(',')[2])
result = rating_values.countByValue()

for rating, number_of_ratings in sorted(result.items()):
    print(f"{number_of_ratings} avaliações tiveram a nota {rating}")

393068 avaliações tiveram a nota 0.5
776815 avaliações tiveram a nota 1.0
399490 avaliações tiveram a nota 1.5
1640868 avaliações tiveram a nota 2.0
1262797 avaliações tiveram a nota 2.5
4896928 avaliações tiveram a nota 3.0
3177318 avaliações tiveram a nota 3.5
6639798 avaliações tiveram a nota 4.0
2200539 avaliações tiveram a nota 4.5
3612474 avaliações tiveram a nota 5.0
CPU times: user 399 ms, sys: 48.7 ms, total: 448 ms
Wall time: 1min 20s


#### Comentários do código
Primeiro é criado um ponteiro para o arquivo CSV ser lido como texto (**Transformation**)
```python
file_csv = sc.textFile(RATINGS_CSV_PATH)
```
As linhas abaixo apenas estão separando a linha do header dos dados (**Transformation**)
```python
header = file_csv.first()
ratings = file_csv.filter(lambda row: row != header)
```
Agora fazemos um map para capturar apenas os valores da terceira coluna como string (**Transformation**)
```python
rating_values = ratings.map(lambda row: row.split(',')[2])
```
Depois usamos o método `countByValue` que retorna um dicionário com os ratings como chave e o count como valor (**Action**). Nesse momento, todas as **Transformations** são executadas
```python
result = rating_values.countByValue()
```
O código abaixo imprime o resultado ordenado pelo primeiro valor das tuplas geradas pelo método `items()`, que são os valores da coluna **rating**
```python
for rating, number_of_ratings in sorted(result.items()):
    print(f"{number_of_ratings} avaliações tiveram a nota {rating}")
```

In [ ]:
# Exemplo de código para imprimir o resultado ordenando pelo count de forma decrescente
count_sort = [(y, x) for x, y in result.items()] # Para isso podemos inverter a posição dos valores das tuplas
for number_of_ratings, rating in sorted(count_sort, reverse=True): # Para que seja decrescente é usado o argumento reverse=True
    print(f"{number_of_ratings} avaliações tiveram a nota {rating}")

6639798 avaliações tiveram a nota 4.0
4896928 avaliações tiveram a nota 3.0
3612474 avaliações tiveram a nota 5.0
3177318 avaliações tiveram a nota 3.5
2200539 avaliações tiveram a nota 4.5
1640868 avaliações tiveram a nota 2.0
1262797 avaliações tiveram a nota 2.5
776815 avaliações tiveram a nota 1.0
399490 avaliações tiveram a nota 1.5
393068 avaliações tiveram a nota 0.5


## Qual é o total de avaliações por valor de rating usando o método MapReduce? (Key-value Pair)

In [ ]:
!head 'drive/MyDrive/data/ml-25m/ratings.csv'

userId,movieId,rating,timestamp
1,296,5.0,1147880044
1,306,3.5,1147868817
1,307,5.0,1147868828
1,665,5.0,1147878820
1,899,3.5,1147868510
1,1088,4.0,1147868495
1,1175,3.5,1147868826
1,1217,3.5,1147878326
1,1237,5.0,1147868839


In [ ]:
%%time

def parse_ratings_as_key(row):
    fields = row.split(',') # Gerar uma lista com os valores da linha
    rating_field = float(fields[2]) # Converter o valor da terceira coluna (rating) para float
    return (rating_field, 1) # Retornar um key-value pair com o valor de rating e o inteiro 1

file_csv = sc.textFile(RATINGS_CSV_PATH)
header = file_csv.first()
ratings = file_csv.filter(lambda row: row != header)
ratings_count = ratings.map(parse_ratings_as_key)
ratings_sum = ratings_count.reduceByKey(lambda x, y: x + y)
results = ratings_sum.collect()
for rating, number_of_ratings in sorted(results):
    print(f"{number_of_ratings} avaliações tiveram a nota {rating}")

393068 avaliações tiveram a nota 0.5
776815 avaliações tiveram a nota 1.0
399490 avaliações tiveram a nota 1.5
1640868 avaliações tiveram a nota 2.0
1262797 avaliações tiveram a nota 2.5
4896928 avaliações tiveram a nota 3.0
3177318 avaliações tiveram a nota 3.5
6639798 avaliações tiveram a nota 4.0
2200539 avaliações tiveram a nota 4.5
3612474 avaliações tiveram a nota 5.0
CPU times: user 474 ms, sys: 73.6 ms, total: 548 ms
Wall time: 1min 38s


#### Comentários do código
Criamos uma função para fazer o parse da string dos dados por coluna e tipo
```python
def parse_ratings_as_key(row):
    fields = row.split(',') # Gerar uma lista com os valores da linha
    rating_field = float(fields[2]) # Converter o valor da terceira coluna (rating) para float
    return (rating_field, 1) # Retornar um key-value pair com o valor de rating e o inteiro 1
```
Fazemos um map para capturar apenas os valores da terceira coluna como `float` em **key-value pairs** (float, 1)
```python
ratings_count = ratings.map(parse_ratings_as_key)
```
Depois usamos o método `reduceByKey` que executa a função passando os **values** de dois **key-value pairs** cujo **key** é o mesmo. O resultado de cada operação segue como argumento junto com o **value** do próximo **key-value pair**.
```python
ratings_sum = ratings_count.reduceByKey(lambda x, y: x + y)
```
Depois usamos o método `collect` (**Action**) que nesse caso retorna uma lista com todos os **key-value pairs** resultantes das **Transformations** acima
```python
results = ratings_sum.collect()
```
O código abaixo imprime o resultado ordenado pelo primeiro valor das tuplas da lista, que são os valores da coluna **rating**
```python
for rating, number_of_ratings in sorted(result):
    print(f"{number_of_ratings} avaliações tiveram a nota {rating}")
```

#### Exemplo de reduceByKey com 1 thread

In [ ]:
(5.0, 1), (5.0, 1), (5.0, 1), (5.0, 1), (5.0, 1), (5.0, 1)
(5.0, 2), (5.0, 1), (5.0, 1), (5.0, 1), (5.0, 1)
(5.0, 3), (5.0, 1), (5.0, 1), (5.0, 1)
(5.0, 4), (5.0, 1), (5.0, 1)
(5.0, 5), (5.0, 1)
(5.0, 6)

(5.0, 6)

#### Exemplo de reduceByKey com 2 threads

In [ ]:
(5.0, 1), (5.0, 1), (5.0, 1), (5.0, 1), (5.0, 1), (5.0, 1)
(5.0, 2), (5.0, 1), (5.0, 2), (5.0, 1)
(5.0, 3), (5.0, 3)
(5.0, 6)

(5.0, 6)

## Qual é a média das avaliações dos 10 primeiros usuários do sistema? (mapValues)

In [ ]:
!head 'drive/MyDrive/data/ml-25m/ratings.csv'

userId,movieId,rating,timestamp
1,296,5.0,1147880044
1,306,3.5,1147868817
1,307,5.0,1147868828
1,665,5.0,1147878820
1,899,3.5,1147868510
1,1088,4.0,1147868495
1,1175,3.5,1147868826
1,1217,3.5,1147878326
1,1237,5.0,1147868839


In [ ]:
%%time

def parse_user_and_rating(row):
    fields = row.split(',')
    user_field = int(fields[0])
    rating_field = float(fields[2])
    return (user_field, rating_field)

file_csv = sc.textFile(RATINGS_CSV_PATH)
header = file_csv.first()
ratings = file_csv.filter(lambda row: row != header)
ratings_by_user = ratings.map(parse_user_and_rating)
totals_by_user_map = ratings_by_user.mapValues(lambda rating: (rating, 1))
totals_by_user = totals_by_user_map.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
ratings_avg = totals_by_user.mapValues(lambda z: z[0] / z[1])
results = ratings_avg.takeOrdered(10)
for user, avg in results:
    print(f"A média de notas do usuário {user} é {avg}")

A média de notas do usuário 1 é 3.8142857142857145
A média de notas do usuário 2 é 3.630434782608696
A média de notas do usuário 3 é 3.6974085365853657
A média de notas do usuário 4 é 3.378099173553719
A média de notas do usuário 5 é 3.7524752475247523
A média de notas do usuário 6 é 4.153846153846154
A média de notas do usuário 7 é 3.64
A média de notas do usuário 8 é 3.6129032258064515
A média de notas do usuário 9 é 3.865168539325843
A média de notas do usuário 10 é 3.452830188679245
CPU times: user 555 ms, sys: 82.4 ms, total: 637 ms
Wall time: 1min 53s


#### Comentários do código
Fazemos um map para capturar apenas os valores da primeira coluna como `int` da terceira coluna como `float` em **key-value pairs** (int, float)
```python
ratings_by_user = ratings.map(parse_user_and_rating)
```
Depois calculamos ao mesmo tempo a soma dos ratings com a quantidade de ratings por usuário. Para isso usamos o método `mapValues` para trocar os **values** dos **key-value pairs** de float por (float, 1). Então usamos o método `reduceByKey` para fazer a soma dos valores de rating e das quantidades por usuário.
```python
totals_by_user = ratings_by_user.mapValues(lambda rating: (rating, 1))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
```
Depois usamos o `mapValues` novamente para calcular a média das notas dos usuários
```python
ratings_avg = totals_by_user.mapValues(lambda z: z[0] / z[1])
```
Depois usamos o método `takeOrdered(10)` (**Action**) que nesse caso retorna uma lista com 10 **key-value pairs** resultantes das **Transformations** acima ordenados pelo **key**
```python
results = ratings_avg.takeOrdered(10)
```

In [ ]:
(1 + 2 + 3 + 4 + 5)

15

In [ ]:
(1 + 2 + 3 + 4 + 5) / 5

3.0

#### Exemplo de reduceByKey no totals_by_user_map

In [ ]:
(1, (1.0, 1)), (1, (2.0, 1)), (1, (3.0, 1)), (1, (4.0, 1)), (1, (5.0, 1))
(1, (3.0, 2)), (1, (3.0, 1)), (1, (4.0, 1)), (1, (5.0, 1))
(1, (6.0, 3)), (1, (4.0, 1)), (1, (5.0, 1))
(1, (10.0, 4)), (1, (5.0, 1))
(1, (15.0, 5))

(1, (15.0, 5))

#### Exemplo de mapValues no totals_by_user

In [ ]:
(1, (15.0, 5)) => (1, 3.0)

## Como testar um código Spark quando eu não tenho um CSV? (parallelize)

In [ ]:
%%time

text = sc.parallelize(["Um texto simples",
                       "Um texto de exemplo",
                       "Exemplo de texto"])
print(type(text))
print(text.take(10))
rdd = text.map(lambda x: x.split())
print(rdd.take(10))
rdd = text.flatMap(lambda x: x.split()) # Experimente usar x.lower().split() para ver o que muda no resultado
print(rdd.take(10))
word_count = rdd.countByValue()

for word, count in word_count.items():
    print(f"{word}: {count}")

<class 'pyspark.rdd.RDD'>
['Um texto simples', 'Um texto de exemplo', 'Exemplo de texto']
[['Um', 'texto', 'simples'], ['Um', 'texto', 'de', 'exemplo'], ['Exemplo', 'de', 'texto']]
['Um', 'texto', 'simples', 'Um', 'texto', 'de', 'exemplo', 'Exemplo', 'de', 'texto']
Um: 2
texto: 3
simples: 1
de: 2
exemplo: 1
Exemplo: 1
CPU times: user 50.8 ms, sys: 3.31 ms, total: 54.1 ms
Wall time: 1.42 s


#### Comentários do código
Usamos o método `parallelize` para carregar um RDD a partir de uma lista Python
```python
text = sc.parallelize(["Um texto simples", "Um texto de exemplo", "Exemplo de texto"])
```

## Como ordenar o resultado pelo count das palavras? (sortByKey)

In [ ]:
%%time

text = sc.parallelize(["Um texto simples",
                       "Um texto exemplo",
                       "Exemplo de texto"])
rdd = text.flatMap(lambda x: x.lower().split())
rdd_counted = rdd.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
rdd_sorted = rdd_counted.map(lambda x: (x[1], x[0])).sortByKey() # Experimente adicionar o argumento ascending=False no sortByKey

word_count = rdd_sorted.collect()
for count, word in word_count:
    print(f"{count}: {word}")

1: simples
1: de
2: exemplo
2: um
3: texto
CPU times: user 46 ms, sys: 2.69 ms, total: 48.7 ms
Wall time: 2.42 s


#### Comentários do código
Usamos o método `sortByKey` para fazer o sort dos dados de maneira distribuída pela **key** dos **key-value pairs**. No momento do sort um `map` executado anteriormente fez uma troca colocando o count das palavras como **key** e as palavras como **value**
```python
rdd_sorted = rdd_counted.map(lambda x: (x[1], x[0])).sortByKey()
```